In [1]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta
from hashlib import sha256
import codecs


from bulletin.data.tb_pacientes import TbPacientes
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer, auto_fit_columns
from bulletin.commom import static
from bulletin.commom.normalize import *

pd.set_option('display.max_columns', None)

output = join("output","correcoes")

if not isdir(output):
    makedirs(output)


In [2]:
casos_confirmados = CasosConfirmados()
tb_pacientes = TbPacientes()
# casos_confirmados.update()
# tb.update()

In [3]:
casosc = casos_confirmados.get_casos()
casost = tb_pacientes.get()
print(f"casos casos_confirmados: {casosc.shape}")
print(f"casos tb_pacientes: {casost.shape}")

casos casos_confirmados: (515439, 23)
casos tb_pacientes: (515439, 48)


In [4]:
casost = casost[['identificacao','status','dt_atualizacao','dt_com_obito','dt_com_recuperado']]

In [5]:
casosc = casosc.rename(columns={'id_access':'identificacao'})
casosc = pd.merge(casosc,casost,how='left',on='identificacao')

In [6]:
casosc[['ordem','identificacao','ibge7','nome','sexo','idade','mun_resid','mun_atend','rs','laboratorio','dt_diag','comunicacao','is','obito','data_obito','status','dt_atualizacao','dt_com_obito','dt_com_recuperado','excluir']]
new_casos = casosc[['ordem','identificacao','ibge7','nome','sexo','idade','mun_resid','mun_atend','rs','laboratorio','dt_diag','comunicacao','is','obito','data_obito','dt_com_obito','status','dt_atualizacao','dt_com_recuperado','excluir']]
new_obitos = casosc.loc[casosc['obito']=='SIM'][['ordem','identificacao','ibge7','nome','sexo','idade','mun_resid','rs','data_obito','dt_com_obito','excluir']]

In [7]:
writer = pd.ExcelWriter(join(output,"Casos confirmados.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

new_casos.to_excel(writer,sheet_name='Casos confirmados',index=None)
worksheet = writer.sheets['Casos confirmados']
auto_fit_columns(worksheet,new_casos)

new_obitos.to_excel(writer,sheet_name='Obitos',index=None)
worksheet = writer.sheets['Obitos']
auto_fit_columns(worksheet,new_obitos)


writer.save()
writer.close()

NameError: name 'auto_fit_columns' is not defined